In [26]:
import os 
import time
import copy
from tqdm import tqdm

import numpy as np 
import matplotlib.pyplot as plt
import scipy
from scipy import sparse
from scipy import linalg
from scipy.linalg import expm, sinm, cosm
import pandas as pd

from joblib import Parallel, delayed
from numba import jit, njit, prange

#################### Macros #############################
np.random.seed(0)

# Physical constants
K=10 # number of fermionic modes <-- They use K=17 in the paper
BETA = 5 # inverse temperature
N = 2*K # number of fermions
N_DIM = 2**K # Hilbert space dimensions
J=4 # ~"energy scale" <-- Their J not given in the paper, nor whether we're even in strong or weak-coupling regime. Maybe all that matters is t*J, which is what they plot?
Q=4 # order of coupling <-- Their q also not given. 4 is pretty generic, so let's stick with that 
E_ORDER = 19 # Order to keep of matrix-exponential expansion. Also not given in the paper. 

# Directory to save sample Hamiltonians
H_DIR = os.path.join("Simulated Hamiltonians", f"H4_K{K}_J{J}_Q{Q}")
os.makedirs(H_DIR, exist_ok=True)

N_SAMPLES = 160 # number of samples to generate
N_JOBS = 20 # number of jobs to run in parallel


## Load pre-computed sample Hamiltonians

See notebook: "computing_eigenvalues_for_unfolding.ipynb"

In [8]:
H_all= np.zeros((N_SAMPLES, N_DIM, N_DIM), dtype=np.complex128) 
#iv_all = np.zeros((N_SAMPLES, N_DIM), dtype=np.float64) # iv = eigenvalues
for i in range(N_SAMPLES):
    H_all[i] = np.load(os.path.join(H_DIR, f"H4_{i+1}.npy"))
    #iv_all[i] = np.linalg.eigvalsh(H_all[i])

iv_all = np.load(os.path.join(H_DIR, "eigenvalues_all.npy"))
#np.save(os.path.join(H_DIR, "eigenvalues_all.npy"), iv_all)

In [9]:
test_index = np.random.choice(N_SAMPLES)
H_test = H_all[test_index]
iv_test = iv_all[test_index]

In [27]:
def matrix_exponential(A, n): # A = matrix, n = order of expansion
    out = np.zeros(A.shape, dtype=np.complex128)
    last_term = np.identity(A.shape[0])
    out += last_term
    for i in range(1, n+1):
        last_term = last_term @ A / i
        out += last_term
        
    return out

tic = time.time()
e_to_H = matrix_exponential(H_test, E_ORDER)
toc = time.time()
duration = toc - tic
print(f"Dense: {duration//60} minutes, {duration%60} seconds")

later = """
def matrix_exponential_sparse(A, n): # A = matrix, n = order of expansion
    out = sparse.csr_array(np.zeros(A.shape, dtype=np.complex128))
    last_term = sparse.csr_array(np.identity(A.shape[0]))
    out += last_term
    for i in range(1, n+1):
        last_term = last_term @ A / i
        out += last_term
        
    return out

H_test_sparse = sparse.csr_array(H_test)
tic = time.time()
e_to_H_sparse = matrix_exponential_sparse(H_test_sparse, E_ORDER)
toc = time.time()
duration = toc - tic
print(f"Sparse: {duration//60} minutes, {duration%60} seconds")


print(np.allclose(e_to_H, e_to_H_sparse.toarray()))"""

Dense: 0.0 minutes, 0.813676118850708 seconds


## Compare how close self-defined matrix-exponential is with scipy function

In [11]:
tic = time.time()
e_to_H_scipy = expm(H_test)
toc = time.time()
duration = toc - tic
print(f"Scipy: {duration//60} minutes, {duration%60} seconds")

Scipy: 0.0 minutes, 2.4136734008789062 seconds


Scipy function too slow

In [28]:
print(f"E_ORDER: {E_ORDER}")
print("allclose: ", np.allclose(e_to_H, e_to_H_scipy))

abs_diff = np.abs(e_to_H - e_to_H_scipy)
print("max_abs_diff: ", np.max(np.max(abs_diff, axis=0), axis=0))
print("sum_abs_diff: ", np.sum(np.sum(abs_diff, axis=0), axis=0))

E_ORDER: 19
allclose:  True
max_abs_diff:  1.0674152228773437e-08
sum_abs_diff:  0.00039559482875329655


We want the sum of the absolute difference to be at least (a few orders of magnitude) smaller than the Hilbert space dimension (4096 for K=12, 1024 for K=10)

For K=12: E_ORDER=13 seems optimal
- E_ORDER=6: sum_abs_diff = 248906, time = 15.4 seconds
- E_ORDER=10: sum_abs_diff = 8483, time = 23.5 seconds
- E_ORDER=12: sum_abs_diff = 970, time = 30 seconds
- E_ORDER=13: sum_abs_diff = 6e-4, time = 31 seconds
- E_ORDER=15: sum_abs_diff = 5e-5, time = 36.6 seconds
- E_ORDER=20: sum_abs_diff = 1.5e-8, time = 58 seconds

For K=10: E_ORDER=19 seems optimal
- E_ORDER=19: sum_abs_diff = 4e-4, time=0.81 seconds

## Quick check to see if sparse matrix-exponential is faster (it's probably not)

In [ ]:
later = """
print(f"E_ORDER: {E_ORDER}")

tic = time.time()
e_to_H_sparse = matrix_exponential_sparse(H_test_sparse, E_ORDER)
toc = time.time()
duration = toc - tic
print(f"Dense: {duration//60} minutes, {duration%60} seconds")

e_to_H_sparse = e_to_H_sparse.toarray()
print("allclose: ", np.allclose(e_to_H_sparse, e_to_H_scipy))

abs_diff = np.abs(e_to_H_sparse - e_to_H_scipy)
print("max_abs_diff: ", np.max(np.max(abs_diff, axis=0), axis=0))
print("sum_abs_diff: ", np.sum(np.sum(abs_diff, axis=0), axis=0))"""

TODO: Not sure *why* it's slower... 
Also has larger error relative to non-sparse version...

## Define spectral form factor, disorder-averaged analogs

## Straightforward version, proof of concept

For a single timestep, each disorder-term requires 2 (sequential?)  matrix exponentials (and 2 traces, a conjugate, etc.). Expect that each timestep should therefore take 2*N_SAMPLES*E_ORDER_TIME (or 1*...), where E_ORDER_TIME is the time it takes to compute a matrix exponential for the given parameters K, E_ORDER, etc...

N_SAMPLES=160 is standard so far
- For K=10, E_ORDER=19: exp_time = 1.61*160 = 257 seconds ~ 4 minutes
- For K=12, E_ORDER=13: exp_time = 31*160 = 4960 seconds ~ 82 minutes!!

In [32]:
def Z(H): # <-- RETURNS SCALAR
    return np.trace(matrix_exponential(-BETA*H, E_ORDER))

def Zt(H,t): # <-- RETURNS SCALAR
    return np.trace(matrix_exponential(-(BETA+1j*t)*H, E_ORDER))

def g(H_all, t): # <-- RETURNS SCALAR
    g_num_all = np.zeros(H_all.shape[0], dtype=np.complex128)
    sqrt_g_den_all = np.zeros(H_all.shape[0], dtype=np.complex128)
    for i in range(H_all.shape[0]):
        H_i = H_all[i]
        Zt_i = Zt(H_i, t)
        g_num_all[i] = Zt_i*np.conj(Zt_i)

        sqrt_g_den_all[i] = Z(H_i)
    
    g_num = np.mean(g_num_all, axis=0)
    g_den = np.mean(sqrt_g_den_all, axis=0)**2
    return g_num/g_den

# test for a single timestep
test_t = 12
tic = time.time()
g_test = g(H_all, test_t)
toc = time.time()
duration = toc - tic
print(f"Single time-step, no parallelization: {duration//60} minutes, {duration%60} seconds")

Single time-step, no parallelization: 11.0 minutes, 29.364864826202393 seconds


In [36]:
print(np.log10(g_test))
print(np.log(test_t*J))

(13.871971220532524-7.366260392482353e-21j)
3.871201010907891


Per figure 1 in the paper (and per common sense), $g<10^0=1$

Taking too long so I quit it

## Parallelized version

Should take as long as sequential version divided by N_JOBS

N_SAMPLES=160, N_JOBS=20 is standard so far
- For K=10, E_ORDER=19: exp_time = 1.61*160/20 = 12.85 seconds 
- For K=12, E_ORDER=13: exp_time = 31*160 = 248 seconds ~ 4 minutes

In [37]:
def Z(H):
    return np.trace(matrix_exponential(-BETA*H, E_ORDER))

def Zt(H,t):
    return np.trace(matrix_exponential(-(BETA+1j*t)*H, E_ORDER))

def g_i(H_all, t, g_num_all, sqrt_g_den_all, i): # Computes Zt(H,t)*Z*(H,t) for a single sample, sends result to g_num_all[i], computes Z(H) for a single sample, sends result to sqrt_g_den_all[i]
    H_i = H_all[i]
    Zt_i = Zt(H_i, t)
    g_num_all[i] = Zt_i*np.conj(Zt_i)
    sqrt_g_den_all[i] = Z(H_i)

def g_par(H_all, t, n_jobs=N_JOBS): # <-- makes more sense to parallelize over samples, not time-steps
    g_num_all = np.zeros(H_all.shape[0], dtype=np.complex128)
    sqrt_g_den_all = np.zeros(H_all.shape[0], dtype=np.complex128)
    Parallel(n_jobs=n_jobs)(delayed(g_i)(H_all, t, g_num_all, sqrt_g_den_all, i) for i in range(H_all.shape[0]))

    g_num = np.mean(g_num_all, axis=0)
    g_den = np.mean(sqrt_g_den_all, axis=0)**2
    return g_num/g_den

# test for a single timestep
tic = time.time()
g_par_test = g_par(H_all, test_t)
toc = time.time()
duration = toc - tic
print(f"Single time-step, with parallelization: {duration//60} minutes, {duration%60} seconds")

In [ ]:
print(np.log10(g_test))
print(np.log(test_t*J))

Check that sequential and parallel version give the same results

In [ ]:
print(g_test, g_par_test)
abs_diff = np.abs(g_test - g_par_test)

For all timesteps...

In [ ]:
t_range_test = np.linspace(0,1,10)
g_range_test = np.zeros(t_range_test.shape, dtype=np.complex128)

tic = time.time()
for i in len(g_range_test):
    t_i = t_range_test[i]
    g_i = g(H_all, t_i)
    g_range_test[i] = g_i
toc = time.time()
duration = time.time() - tic 